In [1]:
import matplotlib.pyplot as plt
from data_loader import load_breast_cancer

In [3]:
bc = load_breast_cancer("breast-cancer", scaled=True)
X, y = bc["x"], bc["y"]

In [4]:
bc

{'x': array([[-0.860107, -0.111111, -1.      , ..., -0.555556, -1.      ,
         -1.      ],
        [-0.859671, -0.111111, -0.333333, ..., -0.555556, -0.777778,
         -1.      ],
        [-0.857807, -0.555556, -1.      , ..., -0.555556, -1.      ,
         -1.      ],
        ...,
        [-0.876716, -0.111111,  1.      , ...,  0.555556,  1.      ,
         -0.777778],
        [-0.875424, -0.333333,  0.555556, ...,  1.      ,  0.111111,
         -1.      ],
        [-0.875424, -0.333333,  0.555556, ...,  1.      , -0.333333,
         -1.      ]], shape=(683, 10)),
 'y': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
        1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0